In [10]:
### basic functions
def getTextFromWeb(url: str, headers: dict, params: dict = None) -> str:
    with requests.session() as session, session.get(url=url, headers=headers, verify=False, params=params) as resp:
        resp.encoding=resp.apparent_encoding
        return resp.text

def getJsonFromWeb(url: str, headers: dict, params: dict = None) -> dict:
    return json.loads(getTextFromWeb(url=url, headers=headers, params=params))

In [11]:
tasks=[]
def getUpAllBvid():
    url=' https://api.bilibili.com/x/space/arc/search'
    global params
    global tasks
    getJson=getJsonFromWeb(url=url,headers=headers,params=params)
    if str(getJson['code']) == '0':
        vlist=getJson['data']['list']['vlist']
        tasks.extend(vlist)
        if len(vlist) < 30:
            print("最后一页")
        else:
            print("不是最后一页")
            params['pn']=params.get('pn')+1
            getUpAllBvid()
            
if __name__ == '__main__':
    getUpAllBvid()
    with open('app.json', 'w') as f:
        json.dump(tasks, f)

/Users/sherrycai/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:981: InsecureRequestWarning:

Unverified HTTPS request is being made to host 'api.bilibili.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings



不是最后一页


/Users/sherrycai/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:981: InsecureRequestWarning:

Unverified HTTPS request is being made to host 'api.bilibili.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings



不是最后一页


/Users/sherrycai/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:981: InsecureRequestWarning:

Unverified HTTPS request is being made to host 'api.bilibili.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings



最后一页


In [1]:
import requests
import re
import os
import sys
import json
from bs4 import BeautifulSoup
import csv
import pandas as pd
from itertools import islice
import numpy as np

from requests.packages.urllib3.exceptions import InsecureRequestWarning
from requests.packages.urllib3 import disable_warnings

In [2]:
headers = {'accept': 'application/json, text/plain, */*',
'accept-encoding': 'utf-8',
'accept-language': 'zh-CN,en;q=0.9',
'cookie': "buvid3=EA910064-72DD-4B05-85A5-794CBCEC5854148802infoc; _uuid=ED1BDE5F-4BDD-3ADE-B336-FA33FE03E60791846infoc; blackside_state=1; rpdid=|(J|)Ylu|mlY0J'uYk~|JY|J); buvid_fp=EA910064-72DD-4B05-85A5-794CBCEC5854148802infoc; SESSDATA=5e282d0e%2C1643482156%2C1d01d%2A81; bili_jct=b6ddfda906b95b19150669899645c342; DedeUserID=360048721; DedeUserID__ckMd5=73f4711ca4104ba6; sid=a6hptr9p; LIVE_BUVID=AUTO9416312806429530; CURRENT_QUALITY=32; fingerprint=e09b8b9b55253e3a75784174b92c64a6; buvid_fp_plain=CC157ECE-EB5B-4960-B56D-25E3E52C54D5148810infoc; bp_video_offset_360048721=584833395912863729; video_page_version=v_old_home_19; PVID=4; CURRENT_BLACKGAP=0; CURRENT_FNVAL=976; bp_t_offset_360048721=586740743645076378; innersign=1; bsource=search_baidu",
'origin': 'https://space.bilibili.com',
'referer': 'https://space.bilibili.com/500146471/video',
'sec-ch-ua': '"Google Chrome";v="95", "Chromium";v="95", ";Not A Brand";v="99"',
'sec-ch-ua-mobile': '?0',
'sec-ch-ua-platform': '"macOS"',
'sec-fetch-dest': 'empty',
'sec-fetch-mode': 'cors',
'sec-fetch-site': 'same-site',
'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.54 Safari/537.36'}
params = {'mid': 500146471,
'ps': 30,
'tid': 0,
'pn': 1,
'keyword': '',
'order': 'pubdate',
'jsonp': 'jsonp'}

In [3]:
def get_csv(cid):   #请求的方式得到数据jason文件
    cid=cid
    urll='https://comment.bilibili.com/'+str(cid)+'.xml'#利用cid获取对应弹幕
    rr=requests.get(url=urll,headers=headers)
    rr.encoding='uft-8'
    soup=BeautifulSoup(rr.text,'lxml')
    danmu_info=soup.find_all('d')
    all_info=[]
    all_text=[]
    for i in danmu_info:
        all_info.append(i['p']) #得到弹幕信息
        all_text.append(i) #得到弹幕内容
    f = open('danmu_info.csv', 'w', encoding='utf-8')
    csv_writer = csv.writer(f)
    csv_writer.writerow(["time", "type", "size", "color", "Unix", "type", "userID", "rowID"]) #弹幕信息就是按这么个顺序排列的
    for i in all_info:
        i=str(i).split(',') #把弹幕信息分隔好
        csv_writer.writerow(i)
    f.close()
    f = open('danmu_text.csv', 'w', encoding='utf-8')
    csv_writer = csv.writer(f)
    csv_writer.writerow(["content"])
    for i in all_text:
        csv_writer.writerow(i)
    f.close()
if __name__=='__main__':
    cid = 424903662
    get_csv(cid)


In [4]:
df1=pd.read_csv('danmu_info.csv', index_col=False)
df2=pd.read_csv('danmu_text.csv', index_col=False)
df1=pd.merge(df1,df2,left_index=True, right_index=True)
filename = "%s.csv" % cid
filepath="/Users/sherrycai/Desktop/bilibili/danmu_data/"
df1.to_csv(filepath+filename)
df1.head()

time  type  size     color        Unix  type.1    userID  \
0   916.427     1    25  16777215  1637135427       0  7a80ffa8   
1   125.451     1    25  16777215  1637064053       0  5b7ec220   
2  1086.712     1    25  16777215  1636895339       0  188df1d0   
3   140.582     1    25  16777215  1636860432       0  624475fc   
4   977.200     1    25  16777215  1636829488       0  5e3b7413   

               rowID          content  
0  57694868127036416            Metoo  
1  57657447485280256          都2021年了  
2  57568992933179392  因为是在两个GPU上分开训练的  
3  57550691558249984               哈哈  
4  57534468019502592           Recall

In [5]:
range = np.arange(0, df1['time'].max(), 10)
print(df1.time)
df=df1.groupby(pd.cut(df1.time, range)).count()
df = df.rename(columns={'time': 'count'})
df = df.reset_index()
df['time']=df['time'].apply(lambda x: x.mid)
df.head()
df=df[['time', 'count']]
filename = "%s.csv" % cid
filepath5="/Users/sherrycai/Desktop/bilibili/freq_timeline/"+filename
df.to_csv(filepath5)

0       916.427
1       125.451
2      1086.712
3       140.582
4       977.200
         ...   
177     121.893
178      76.034
179     417.831
180     747.012
181     652.027
Name: time, Length: 182, dtype: float64


In [6]:
import plotly.express as px
import plotly
fig = px.line(df, x="time", y="count", title="Number of Comments in Video Timeline") 
filename = "%s.png" % cid
filepath4="/Users/sherrycai/Desktop/bilibili/freq_timeline/"+filename
fig.show()


In [7]:
import jieba
from wordcloud import WordCloud
import matplotlib.pyplot as plt
danmu = [' '.join(df1['content'])][0]
filename = "%s.txt" % cid
filepath3="/Users/sherrycai/Desktop/bilibili/danmu_text/"+filename
with open(filepath3, 'w') as f:
    f.writelines(danmu)

In [8]:
import collections
def stopwordslist(filepath):
    stopwords = [line.strip() for line in open(filepath, 'r', encoding='utf-8').readlines()]
    return stopwords

def jiebaword(text):
    # 加载自定义词典
    jieba.load_userdict(os.getcwd() + os.sep + 'userdict.txt')

    # 加载停用词
    # path = os.getcwd() + os.sep
    stoppath = 'stopword.txt'

    # 过滤文本标点符号
    te_text = re.sub(r'[^\w\s]','',text)

    # 分词
    data = [i for i in jieba.cut(te_text) if i not in stopwordslist(stoppath) and i != ' ']
    # 使用collections统计词频
    count_data = collections.Counter(list(data))
    # 将结果转成dataframe
    df = pd.DataFrame(count_data.items(), columns=['key', 'cnt'])

    return df

if __name__=='__main__':
    fenci=jiebaword(danmu)
    fenci.head()

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/3_/mfkldjrs29qgzg42nkm8q1q80000gn/T/jieba.cache
Loading model cost 0.437 seconds.
Prefix dict has been built successfully.


FileNotFoundError: [Errno 2] No such file or directory: '/Users/sherrycai/Desktop/bilibili/userdict.txt'

In [ ]:
filepath2="/Users/sherrycai/Desktop/bilibili/word_freq/"
filename = "%s.csv" % cid
fenci.to_csv(filepath2+filename)

In [ ]:
danmu1 = " ".join(jieba.cut(danmu))
wordcloud = WordCloud(font_path="simsun.ttf").generate(danmu1)
%pylab inline
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")